In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth

import requests

In [4]:
if not Path(DOWNLOAD_FOLDER_NAME).exists():
    Path(DOWNLOAD_FOLDER_NAME).mkdir()

In [5]:
class Webdriver_Handler:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('disable-infobars')
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        prefs = {
            "profile.default_content_settings.popups": 0,    
            "download.default_directory": str(Path(os.getcwd(), DOWNLOAD_FOLDER_NAME)), ### Set the path accordingly
            "download.prompt_for_download": False, ## change the downpath accordingly
            "download.directory_upgrade": True
        }
        options.add_experimental_option("prefs", prefs)

        self.timeout = 3       

        self.wd = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
        
        self.wd.implicitly_wait(self.timeout)

        stealth(
            self.wd,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )
    
    def get_plenarprotokolle(self):
        self.wd.get(BASE_URL)

        download_divs = self.wd.find_elements(By.XPATH, "//div[@class='container-fluid bt-module-content-wrap']")

        download_divs_titles = [div.find_element(By.XPATH, ".//h2").text for div in download_divs]
        print(f"Possible download divs: {[h2 for h2 in download_divs_titles]}")
        slide_arrows = [div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']") for div in download_divs]
        print("slideable:", [slide_arrow.get_attribute("aria-disabled") == "false" for slide_arrow in slide_arrows])
        xml_links = [div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""") for div in download_divs]
        print("number_xml", [len(xml_link) for xml_link in xml_links])

        for download_div in download_divs:
            div_header = download_div.find_element(By.XPATH, ".//h2").text
            print("Fetching links for", div_header)
            all_download_links = []
            
            link_elements = download_div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""")
            all_download_links += [link_element.get_attribute("href") for link_element in link_elements]

            while len(download_div.find_elements(By.XPATH, ".//button[@class='slick-next slick-arrow']")) > 0:
                if not download_div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']").get_attribute("aria-disabled") == "false":
                    print("Slide button exists but is not clickable")
                    break
                download_div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']").click()
                time.sleep(1)
                link_elements = download_div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""")
                all_download_links += [link_element.get_attribute("href") for link_element in link_elements]

            all_download_links = list(set(all_download_links))

            print(f"All download links for {div_header}: {all_download_links}")
            self.download_all_to_folder(all_download_links, div_header)


    def download_all_to_folder(self, download_links, folder_name):
        s = requests.Session()
        if not Path(DOWNLOAD_FOLDER_NAME, folder_name).exists():
            Path(DOWNLOAD_FOLDER_NAME, folder_name).mkdir()
        
        for download_link in download_links:
            file_name = download_link.split("/")[-1]
            if "?" in file_name:
                file_name = file_name.split("?")[-1]
            if not file_name.endswith(".xml"):
                file_name += ".xml"

            if Path(DOWNLOAD_FOLDER_NAME, folder_name, file_name).exists():
                print(f"File {file_name} already exists")
                continue
            
            print(f"Downloading {file_name}")
            r = s.get(download_link, allow_redirects=True)
            while "Rate-Limit exceeded" in str(r.content):
                print("Rate-Limit exceeded")
                time.sleep(5)
                r = s.get(download_link, allow_redirects=True)
            
            open(Path(DOWNLOAD_FOLDER_NAME, folder_name, file_name), 'wb').write(r.content)
        print(f"Finished downloading {folder_name}")

In [7]:
for folder in os.listdir(DOWNLOAD_FOLDER_NAME):
    for file in os.listdir(Path(DOWNLOAD_FOLDER_NAME, folder)):
        if file.endswith(".xml"):
            s = open(Path(DOWNLOAD_FOLDER_NAME, folder, file), "r", encoding="utf-8").read()
            if "Rate-Limit exceeded" in s:
                print("Rate-Limit exceeded in", folder, file)
                os.remove(Path(DOWNLOAD_FOLDER_NAME, folder, file))